In [2]:
!nvidia-smi

Sat Feb 22 02:00:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.16              Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:15:00.0  On |                  Off |
| 30%   45C    P2             66W /  230W |    5598MiB /  24564MiB |     35%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install flash-attn --no-build-isolation

In [44]:
from PIL import Image 
from transformers import AutoProcessor, AutoModelForImageTextToText 
import torch 

model_path = "HuggingFaceTB/SmolVLM2-2.2B-Instruct" 
processor = AutoProcessor.from_pretrained(model_path) 
model = AutoModelForImageTextToText.from_pretrained(model_path, torch_dtype=torch.bfloat16, _attn_implementation="flash_attention_2").to("cuda") 


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


Sat Feb 22 01:45:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.16              Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:15:00.0  On |                  Off |
| 30%   43C    P8             16W /  230W |    5556MiB /  24564MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
messages = [{ "role": "user", "content": [ {"type": "video", "path": "Music.mp4"}, {"type": "text", "text": "Describe this video in detail"} ] },]

inputs = processor.apply_chat_template( messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt", ).to(model.device, torch.bfloat16) 



In [4]:
generated_ids = model.generate(**inputs, do_sample=False, max_new_tokens=564) 

In [5]:
generated_texts = processor.batch_decode( generated_ids, skip_special_tokens=True, )

In [6]:
generated_texts

['User: You are provided the following series of sixty-four frames from a 0:02:22 [H:MM:SS] video.\n\nFrame from 00:00:\nFrame from 00:03:\nFrame from 00:05:\nFrame from 00:07:\nFrame from 00:09:\nFrame from 00:12:\nFrame from 00:14:\nFrame from 00:16:\nFrame from 00:18:\nFrame from 00:21:\nFrame from 00:23:\nFrame from 00:25:\nFrame from 00:27:\nFrame from 00:29:\nFrame from 00:32:\nFrame from 00:34:\nFrame from 00:36:\nFrame from 00:38:\nFrame from 00:41:\nFrame from 00:43:\nFrame from 00:45:\nFrame from 00:47:\nFrame from 00:50:\nFrame from 00:52:\nFrame from 00:54:\nFrame from 00:56:\nFrame from 00:58:\nFrame from 01:01:\nFrame from 01:03:\nFrame from 01:05:\nFrame from 01:07:\nFrame from 01:10:\nFrame from 01:12:\nFrame from 01:14:\nFrame from 01:16:\nFrame from 01:18:\nFrame from 01:21:\nFrame from 01:23:\nFrame from 01:25:\nFrame from 01:27:\nFrame from 01:30:\nFrame from 01:32:\nFrame from 01:34:\nFrame from 01:36:\nFrame from 01:39:\nFrame from 01:41:\nFrame from 01:43:\nFrame

In [83]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForImageTextToText
from transformers.image_utils import load_image 

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [84]:
# ("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")

In [68]:
# Load images
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")
image3=  Image.open('invoice.png').convert('RGB')

prompt="""
Extract the item names from this invoice. please do not include unit price and total amount. provide the name in list
"""


In [76]:
# Initialize processor and model
model_path = "HuggingFaceTB/SmolVLM2-2.2B-Instruct" 
processor = AutoProcessor.from_pretrained(model_path)

In [77]:
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.34it/s]


In [78]:
# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Can you describe the three images?"}
        ]
    },
]

In [79]:
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2, image3], return_tensors="pt")
inputs = inputs.to(model.device, torch.bfloat16) 

# Generate outputs
generated_ids = model.generate(**inputs, do_sample=False, max_new_tokens=500) 
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)


In [80]:
print(generated_texts[0])

User:












Can you describe the three images?
Assistant: The first image is a photograph of the Statue of Liberty in New York City, with a city skyline in the background. The second image is a photograph of a bee on a pink flower, with other flowers in the background. The third image is an invoice from a company called East Repair Inc., with a list of items and their prices.
